<a href="https://colab.research.google.com/github/flaghunter21/DFS/blob/main/Small_Slate_MLB_Combo_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import itertools
from itertools import combinations
import numpy as np
import sklearn
from sklearn import linear_model

In [4]:
df = pd.read_csv('625_Afternoon.csv')
df.head()

,Player,Inj,Likes,Pos,Salary,Order,Hand,Team,Opp,Time,...,pGrade,Proj,My Proj,Exp.,Used,Con.,Value,Own%,Actual Score,Actual Val
0,Lance Lynn,NaN,NaN,P,7600,SP,R,CWS,vs BAL,1:10 PM,...,B,16.898,16.898,100,NaN,0,2.2,81.37%,15.6,2.1
1,Miles Mikolas,NaN,NaN,P,9600,SP,R,STL,vs CHC,1:15 PM,...,B-,14.997,14.997,100,100%,50,1.6,83.26%,26.5,2.8
2,Cedric Mullins,NaN,NaN,OF,4800,1,L,BAL,@ CWS,1:10 PM,...,A,9.619,9.619,100,33.30%,54,2.0,26.41%,8.0,1.7
3,Luis Robert,NaN,NaN,OF,5600,3,R,CWS,vs BAL,1:10 PM,...,A,10.215,10.215,100,NaN,61,1.8,56.01%,8.0,1.4
4,Paul Goldschmidt,NaN,NaN,1B,6000,3,R,STL,vs CHC,1:15 PM,...,A,10.039,10.039,100,33.30%,60,1.7,43.10%,8.0,1.3


In [5]:
print(df.columns)

Index(['Player', 'Inj', 'Likes', 'Pos', 'Salary', 'Order', 'Hand', 'Team',
       'Opp', 'Time', 'vs. Pitcher', 'Win %', 'Total', 'TeamRuns', 'OppRuns',
       'ERA/AVG', 'wOBA', 'ISO', 'Avg innings', 'K/9', 'WHIP', 'Avg', 'STDV',
       'Ceiling', 'Own', 'vGrade', 'pGrade', 'Proj', 'My Proj', 'Exp.', 'Used',
       'Con.', 'Value', 'Own%', 'Actual Score', 'Actual Val'],
      dtype='object')


In [6]:
df = df.loc[:,['Player', 'Pos', 'Salary','Team','Proj','Own','Actual Score']]
df.head()

,Player,Pos,Salary,Team,Proj,Own,Actual Score
0,Lance Lynn,P,7600,CWS,16.898,96.20%,15.6
1,Miles Mikolas,P,9600,STL,14.997,90.30%,26.5
2,Cedric Mullins,OF,4800,BAL,9.619,30.50%,8.0
3,Luis Robert,OF,5600,CWS,10.215,45.90%,8.0
4,Paul Goldschmidt,1B,6000,STL,10.039,29.90%,8.0


In [7]:
df['Own'] = df['Own'].str.replace('%','').astype(float)/100
df.head()

,Player,Pos,Salary,Team,Proj,Own,Actual Score
0,Lance Lynn,P,7600,CWS,16.898,0.962,15.6
1,Miles Mikolas,P,9600,STL,14.997,0.903,26.5
2,Cedric Mullins,OF,4800,BAL,9.619,0.305,8.0
3,Luis Robert,OF,5600,CWS,10.215,0.459,8.0
4,Paul Goldschmidt,1B,6000,STL,10.039,0.299,8.0


In [8]:
# Pitchers
pitchers = df[df['Pos'] == 'P']['Player']
pitchers

0          Lance Lynn
1       Miles Mikolas
9      Adrian Sampson
33    Spenser Watkins
Name: Player, dtype: object

In [9]:
# Pitcher combinations
pitcher_combos = list(itertools.combinations(pitchers, 2))

In [10]:
pitcher_combos

[('Lance Lynn', 'Miles Mikolas'),
 ('Lance Lynn', 'Adrian Sampson'),
 ('Lance Lynn', 'Spenser Watkins'),
 ('Miles Mikolas', 'Adrian Sampson'),
 ('Miles Mikolas', 'Spenser Watkins'),
 ('Adrian Sampson', 'Spenser Watkins')]

In [11]:
df['Pos'].unique()

array(['P', 'OF', '1B', '2B', '1B/OF', 'SS', '3B', 'C', '2B/SS', '2B/3B'],
      dtype=object)

In [12]:
catchers = df[df['Pos'] == 'C']['Player']
first_base = df[df['Pos'].isin(['1B','1B/OF'])]['Player']
second_base = df[df['Pos'].isin(['2B','2B/SS','2B/3B'])]['Player']
third_base = df[df['Pos'].isin(['3B','2B/3B'])]['Player']
shortstop = df[df['Pos'].isin(['SS','2B/SS'])]['Player']
outfielders = df[df['Pos'].isin(['OF','1B/OF'])]['Player']

In [13]:
infield_combos = list(itertools.product(catchers, first_base, second_base, third_base, shortstop))
outfield_combos = list(itertools.combinations(outfielders, 3))

In [14]:
infield_combos[:5]

[('Willson Contreras',
  'Paul Goldschmidt',
  'Nolan Gorman',
  'Nolan Arenado',
  'Tommy Edman'),
 ('Willson Contreras',
  'Paul Goldschmidt',
  'Nolan Gorman',
  'Nolan Arenado',
  'Lenyn Sosa'),
 ('Willson Contreras',
  'Paul Goldschmidt',
  'Nolan Gorman',
  'Nolan Arenado',
  'Jorge Mateo'),
 ('Willson Contreras',
  'Paul Goldschmidt',
  'Nolan Gorman',
  'Nolan Arenado',
  'Nico Hoerner'),
 ('Willson Contreras',
  'Paul Goldschmidt',
  'Nolan Gorman',
  'Nolan Arenado',
  'Jonathan Arauz')]

In [15]:
outfield_combos[:5]

[('Cedric Mullins', 'Luis Robert', 'Andrew Vaughn'),
 ('Cedric Mullins', 'Luis Robert', 'A.J. Pollock'),
 ('Cedric Mullins', 'Luis Robert', 'Austin Hays'),
 ('Cedric Mullins', 'Luis Robert', 'Gavin Sheets'),
 ('Cedric Mullins', 'Luis Robert', 'Trey Mancini')]

In [16]:
lineup_combos = list(itertools.product(pitcher_combos, infield_combos, outfield_combos))
len(lineup_combos)

11943750

In [17]:
lineup_combos[:3]

[(('Lance Lynn', 'Miles Mikolas'),
  ('Willson Contreras',
   'Paul Goldschmidt',
   'Nolan Gorman',
   'Nolan Arenado',
   'Tommy Edman'),
  ('Cedric Mullins', 'Luis Robert', 'Andrew Vaughn')),
 (('Lance Lynn', 'Miles Mikolas'),
  ('Willson Contreras',
   'Paul Goldschmidt',
   'Nolan Gorman',
   'Nolan Arenado',
   'Tommy Edman'),
  ('Cedric Mullins', 'Luis Robert', 'A.J. Pollock')),
 (('Lance Lynn', 'Miles Mikolas'),
  ('Willson Contreras',
   'Paul Goldschmidt',
   'Nolan Gorman',
   'Nolan Arenado',
   'Tommy Edman'),
  ('Cedric Mullins', 'Luis Robert', 'Austin Hays'))]

In [18]:
combos_df = pd.DataFrame(lineup_combos,columns=['P','IF','OF'])

In [19]:
combos_df.head()

,P,IF,OF
0,"(Lance Lynn, Miles Mikolas)","(Willson Contreras, Paul Goldschmidt, Nolan Go...","(Cedric Mullins, Luis Robert, Andrew Vaughn)"
1,"(Lance Lynn, Miles Mikolas)","(Willson Contreras, Paul Goldschmidt, Nolan Go...","(Cedric Mullins, Luis Robert, A.J. Pollock)"
2,"(Lance Lynn, Miles Mikolas)","(Willson Contreras, Paul Goldschmidt, Nolan Go...","(Cedric Mullins, Luis Robert, Austin Hays)"
3,"(Lance Lynn, Miles Mikolas)","(Willson Contreras, Paul Goldschmidt, Nolan Go...","(Cedric Mullins, Luis Robert, Gavin Sheets)"
4,"(Lance Lynn, Miles Mikolas)","(Willson Contreras, Paul Goldschmidt, Nolan Go...","(Cedric Mullins, Luis Robert, Trey Mancini)"


In [20]:
combos_df[['SP1', 'SP2']] = pd.DataFrame(combos_df['P'].to_list())
combos_df[['C','1B','2B','3B','SS']] = pd.DataFrame(combos_df['IF'].to_list())
combos_df[['OF1', 'OF2','OF3']] = pd.DataFrame(combos_df['OF'].to_list())

In [21]:
combos_df = combos_df.drop(['P','IF','OF'],axis=1)
combos_df

,SP1,SP2,C,1B,2B,3B,SS,OF1,OF2,OF3
0,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Andrew Vaughn
1,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,A.J. Pollock
2,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Austin Hays
3,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Gavin Sheets
4,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Trey Mancini
...,...,...,...,...,...,...,...,...,...,...
11943745,Adrian Sampson,Spenser Watkins,Robinson Chirinos,Alfonso Rivas,Jonathan Arauz,Tyler Nevin,Jonathan Arauz,Juan Yepez,Nelson Velazquez,Ryan McKenna
11943746,Adrian Sampson,Spenser Watkins,Robinson Chirinos,Alfonso Rivas,Jonathan Arauz,Tyler Nevin,Jonathan Arauz,Dylan Carlson,Rafael Ortega,Nelson Velazquez
11943747,Adrian Sampson,Spenser Watkins,Robinson Chirinos,Alfonso Rivas,Jonathan Arauz,Tyler Nevin,Jonathan Arauz,Dylan Carlson,Rafael Ortega,Ryan McKenna
11943748,Adrian Sampson,Spenser Watkins,Robinson Chirinos,Alfonso Rivas,Jonathan Arauz,Tyler Nevin,Jonathan Arauz,Dylan Carlson,Nelson Velazquez,Ryan McKenna


In [22]:
df['Pos'].unique()

array(['P', 'OF', '1B', '2B', '1B/OF', 'SS', '3B', 'C', '2B/SS', '2B/3B'],
      dtype=object)

In [23]:
projection_map = dict(zip(df['Player'], df['Proj']))
ownership_map = dict(zip(df['Player'], df['Own']))
team_map = dict(zip(df['Player'], df['Team']))
salary_map = dict(zip(df['Player'], df['Salary']))
actual_map = dict(zip(df['Player'], df['Actual Score']))

In [24]:
proj_cols = []
own_cols = []
team_cols = []
salary_cols = []
actual_cols = []
combos_proj_df = pd.DataFrame()
combos_own_df = pd.DataFrame()
combos_team_df = pd.DataFrame()
combos_salary_df = pd.DataFrame()
combos_actual_df = pd.DataFrame()

In [25]:
for pos in combos_df.columns:
  combos_proj_df[f'{pos}_Proj'] = combos_df[f'{pos}'].map(projection_map)
  proj_cols.append(f'{pos}_Proj')
combos_proj_df['Total Proj'] = combos_proj_df[proj_cols].sum(axis=1)

In [26]:
for pos in combos_df.columns:
  combos_own_df[f'{pos}_Own'] = combos_df[f'{pos}'].map(ownership_map)
  own_cols.append(f'{pos}_Own')
combos_own_df['Own Sum'] = combos_own_df[own_cols].sum(axis=1)
combos_own_df['Own Product'] = combos_own_df['SP1_Own'] * combos_own_df['SP2_Own']* combos_own_df['C_Own'] * combos_own_df['1B_Own'] * combos_own_df['2B_Own'] * combos_own_df['3B_Own'] * combos_own_df['SS_Own'] * combos_own_df['OF1_Own'] * combos_own_df['OF2_Own'] * combos_own_df['OF3_Own']

In [27]:
for pos in combos_df.columns:
  combos_team_df[f'{pos}_Team'] = combos_df[f'{pos}'].map(team_map).astype('category')
  team_cols.append(f'{pos}_Team')
hitter_cols = np.array(['C_Team', '1B_Team', '2B_Team', '3B_Team', 'SS_Team', 'OF1_Team', 'OF2_Team', 'OF3_Team'])
# How to make this faster? (2 hrs using value_counts)
#stacks = combos_team_df[hitter_cols].apply(pd.Series.value_counts, axis=1).fillna(0)
#combos_team_df['Stack Size'] = stacks.max(axis=1)

In [28]:
for pos in combos_df.columns:
  combos_salary_df[f'{pos}_Salary'] = combos_df[f'{pos}'].map(salary_map)
  salary_cols.append(f'{pos}_Salary')
combos_salary_df['Salary Cap'] = combos_salary_df[salary_cols].sum(axis=1)

In [29]:
for pos in combos_df.columns:
  combos_actual_df[f'{pos}_Actual'] = combos_df[f'{pos}'].map(actual_map)
  actual_cols.append(f'{pos}_Actual')
combos_actual_df['Total Actual'] = combos_actual_df[actual_cols].sum(axis=1)

In [30]:
combos_df = pd.concat([combos_df, combos_proj_df, combos_own_df, combos_team_df, combos_salary_df, combos_actual_df], axis=1)

In [31]:
combos_df.head()

,SP1,SP2,C,1B,2B,3B,SS,OF1,OF2,OF3,...,SP2_Actual,C_Actual,1B_Actual,2B_Actual,3B_Actual,SS_Actual,OF1_Actual,OF2_Actual,OF3_Actual,Total Actual
0,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Andrew Vaughn,...,26.5,7.0,8.0,0.0,15.0,10.0,8.0,8.0,2.0,100.1
1,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,A.J. Pollock,...,26.5,7.0,8.0,0.0,15.0,10.0,8.0,8.0,0.0,98.1
2,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Austin Hays,...,26.5,7.0,8.0,0.0,15.0,10.0,8.0,8.0,14.0,112.1
3,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Gavin Sheets,...,26.5,7.0,8.0,0.0,15.0,10.0,8.0,8.0,10.0,108.1
4,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Cedric Mullins,Luis Robert,Trey Mancini,...,26.5,7.0,8.0,0.0,15.0,10.0,8.0,8.0,9.0,107.1


In [32]:
df['Pos'].unique()

array(['P', 'OF', '1B', '2B', '1B/OF', 'SS', '3B', 'C', '2B/SS', '2B/3B'],
      dtype=object)

In [33]:
### Impose lineup restrictions

# Salary Cap <= $50,000
combos_df = combos_df[combos_df['Salary Cap'] <= 50000]

# Hitter Stack Size <= 5
#combos_df = combos_df[combos_df['Stack Size'] <= 5]

# Address multi-position eligibility (same player used more than once in lineup)
combos_df['Total Proj'] = np.where(combos_df['1B'] == combos_df['OF1'],0,combos_df['Total Proj'])
combos_df['Total Proj'] = np.where(combos_df['1B'] == combos_df['OF2'],0,combos_df['Total Proj'])
combos_df['Total Proj'] = np.where(combos_df['1B'] == combos_df['OF3'],0,combos_df['Total Proj'])
combos_df['Total Proj'] = np.where(combos_df['2B'] == combos_df['3B'],0,combos_df['Total Proj'])
combos_df['Total Proj'] = np.where(combos_df['2B'] == combos_df['SS'],0,combos_df['Total Proj'])

In [34]:
combos_df.head()

,SP1,SP2,C,1B,2B,3B,SS,OF1,OF2,OF3,...,SP2_Actual,C_Actual,1B_Actual,2B_Actual,3B_Actual,SS_Actual,OF1_Actual,OF2_Actual,OF3_Actual,Total Actual
369,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Tommy Edman,Gavin Sheets,Rafael Ortega,Ryan McKenna,...,26.5,7.0,8.0,0.0,15.0,10.0,10.0,16.0,0.0,108.1
504,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Lenyn Sosa,Cedric Mullins,Gavin Sheets,Harrison Bader,...,26.5,7.0,8.0,0.0,15.0,2.0,8.0,10.0,9.0,101.1
507,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Lenyn Sosa,Cedric Mullins,Gavin Sheets,Rafael Ortega,...,26.5,7.0,8.0,0.0,15.0,2.0,8.0,10.0,16.0,108.1
508,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Lenyn Sosa,Cedric Mullins,Gavin Sheets,Nelson Velazquez,...,26.5,7.0,8.0,0.0,15.0,2.0,8.0,10.0,6.0,98.1
509,Lance Lynn,Miles Mikolas,Willson Contreras,Paul Goldschmidt,Nolan Gorman,Nolan Arenado,Lenyn Sosa,Cedric Mullins,Gavin Sheets,Ryan McKenna,...,26.5,7.0,8.0,0.0,15.0,2.0,8.0,10.0,0.0,92.1


In [48]:
final_combos = combos_df.loc[:,['Total Proj', 'Own Sum', 'Own Product', 'Total Actual', 'Salary Cap']]
final_combos = final_combos[final_combos['Total Proj'] > 0]
final_combos.sort_values(by='Total Actual', ascending=False).head()

,Total Proj,Own Sum,Own Product,Total Actual,Salary Cap
9806456,77.935,2.575,5.955205e-08,151.15,40900
8213956,79.730,2.794,1.464981e-07,151.15,43700
8213982,79.921,2.788,1.419672e-07,150.15,44200
9806489,78.096,2.577,6.016599e-08,150.15,42100
9806482,78.126,2.569,5.771024e-08,150.15,41400


In [49]:
X = final_combos[['Total Proj','Own Sum']]
y = final_combos['Total Actual']

In [50]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [51]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 48.323676255541656
Coefficients: 
 [ 0.53281668 -2.32835147]
